In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from keras import backend as K

classification = {
    0: "nsfw",
    1: "sfw"
}

Using TensorFlow backend.


In [10]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

img_width , img_height = 255,255

if K.image_data_format() == 'channels_first':
    shape = (3, img_width, img_height)
else:
    shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [116]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2)

test_datagen = ImageDataGenerator(
    rescale=1. / 255
)

predict_datagen = ImageDataGenerator()

In [138]:
train_data_dir = './dataset/train_set'
validation_data_dir = './dataset/validation_set'
predict_data_dir = './dataset/predict_set'
epochs = 5
batch_size = 32

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

prediction_generator = predict_datagen.flow_from_directory(
    predict_data_dir,
    target_size=(img_width, img_height),
    shuffle=False,
    class_mode='binary')

Found 6266 images belonging to 2 classes.
Found 1302 images belonging to 2 classes.
Found 13 images belonging to 2 classes.


In [17]:
model.fit_generator(
    train_generator,
    steps_per_epoch=6266 // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=1302 // batch_size
)

Epoch 1/5
195/195 [==============================] - 205s 1s/step - loss: 0.4642 - accuracy: 0.8069 - val_loss: 1.9648 - val_accuracy: 0.6539
Epoch 2/5
195/195 [==============================] - 205s 1s/step - loss: 0.3489 - accuracy: 0.8720 - val_loss: 0.2082 - val_accuracy: 0.9157
Epoch 3/5
195/195 [==============================] - 204s 1s/step - loss: 0.3255 - accuracy: 0.8823 - val_loss: 0.0501 - val_accuracy: 0.9346
Epoch 4/5
195/195 [==============================] - 206s 1s/step - loss: 0.3195 - accuracy: 0.8877 - val_loss: 0.2353 - val_accuracy: 0.9331
Epoch 5/5
195/195 [==============================] - 207s 1s/step - loss: 0.2997 - accuracy: 0.8920 - val_loss: 0.1487 - val_accuracy: 0.9394


In [121]:
test_data_dir = './dataset/test_set'
test_it = test_datagen.flow_from_directory(
    test_data_dir, 
    target_size=(img_width, img_height), 
    class_mode='binary', 
    batch_size=batch_size
)
result = model.evaluate_generator(test_it, steps=32)
print('loss:', result[0], 'accuracy:', result[1])

Found 1299 images belonging to 2 classes.
loss: 0.10353092849254608 accuracy: 0.93359375


In [139]:
pred = model.predict(prediction_generator)
pred

array([[1.        ],
       [0.        ],
       [0.01990941],
       [0.9999987 ],
       [1.        ],
       [1.        ],
       [0.        ],
       [0.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ],
       [1.        ]], dtype=float32)

In [140]:
img = load_img(predict_data_dir + '/nsfw/10.jpg')
data = img_to_array(img)
data = data.reshape((1,) + data.shape)
pred = model.predict(data)
pred

array([[0.]], dtype=float32)

In [122]:
print(train_generator.class_indices)

{'nsfw': 0, 'sfw': 1}
